In [2]:
# !pip install -r requirements.txt
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib
import re
import nltk
import numpy as np
from IPython.display import Image
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
dati_aprile = pd.read_csv('Job_vacancies_aprile_completo.csv', skiprows=1, usecols = [1,2,3],\
                          names = ['Target','Sub_Role','Job_Description'])
dati_aprile = dati_aprile.reindex(np.random.permutation(dati_aprile.index))
print('dati_aprile shape {}'.format(dati_aprile.shape))
dati_aprile[['Target','Job_Description']].head() # per facilitare la comprensione dei dati di testo da parte del modello

dati_aprile shape (650, 3)


,Target,Job_Description
645,Commessi delle vendite al minuto,General s.r.l. proprietaria del marchio BACCI ...
628,Commessi delle vendite al minuto,Prestigiosa azienda nel settore orologeria / g...
476,Camerieri di ristorante,Siamo alla ricerca di una risorsa da inserire ...
213,Statistici,La risorsa si occuperà in team delle seguenti ...
512,Commessi delle vendite al minuto,Titolo di studio minimo Licenza media\n ...


### FastText Word Embeddings

In [5]:
import fasttext
import fasttext.util

In [9]:
fasttext.util.download_model('it', if_exists='ignore')

 (100.00%) [==================================================>]]=================================================> ]=================================================> ]]=================================================> ]=================================================> ]=================================================> ]=================================================> ]> ]] ]=================================================> ]=================================================> ]=================================================> ]=================================================> ] ]> ]=================================================> ] ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]> ]==================================

'cc.it.300.bin'

In [6]:
ft = fasttext.load_model('cc.it.300.bin')

In [ ]:
ft.get_analogies('re','uomo','donna')

In [10]:
dati_aprile['word_embedding'] = dati_aprile.Job_Description.apply(lambda x: ft.get_sentence_vector(x.replace('\n',' ')))



In [180]:
data = dati_aprile[['word_embedding','Target']]
data.head()

,word_embedding,Target
645,"[0.02905994, 0.0019087426, 0.031447142, 0.0015...",Commessi delle vendite al minuto
628,"[0.014807243, -0.0033119374, 0.044902157, 0.00...",Commessi delle vendite al minuto
476,"[0.014254641, -0.0135342125, 0.041607685, 0.00...",Camerieri di ristorante
213,"[0.022764275, -0.00050031685, 0.04481152, 0.00...",Statistici
512,"[0.02147509, -0.009274839, 0.020515963, 0.0002...",Commessi delle vendite al minuto


In [181]:
data.word_embedding = data.word_embedding.apply(lambda x: ','.join(str(i) for i in np.split(x,300)))
matrix_embedding = data.word_embedding.str.split(',',expand = True)

/opt/anaconda3/envs/giorgio/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [182]:
dati_with_embedd = pd.DataFrame(np.concatenate([data,matrix_embedding], axis = 1))

In [183]:
dati_with_embedd.drop([0], axis = 1, inplace = True)
dati_with_embedd.head()

,1,2,3,4,5,6,7,8,9,10,...,292,293,294,295,296,297,298,299,300,301
0,Commessi delle vendite al minuto,[0.02905994],[0.00190874],[0.03144714],[0.00154232],[-0.00390742],[-0.00678748],[0.00961656],[-0.0099149],[-0.00947997],...,[-0.01230825],[-0.00669219],[-0.00417595],[0.009623],[0.03500751],[-0.00021694],[0.01298142],[-0.01901937],[0.00634783],[-0.00222997]
1,Commessi delle vendite al minuto,[0.01480724],[-0.00331194],[0.04490216],[0.00101441],[-0.00983409],[0.00302889],[0.00228264],[0.00314353],[-0.03837845],...,[-0.02145843],[-0.01862207],[0.00790173],[0.01669352],[0.00919823],[0.01074163],[-0.00850539],[-0.01759769],[-0.00131906],[0.00105229]
2,Camerieri di ristorante,[0.01425464],[-0.01353421],[0.04160769],[0.0056012],[-0.00887701],[-0.00867976],[0.01096454],[-0.00535406],[-0.03199022],...,[-0.01336798],[-0.01496583],[-0.00165584],[0.00823019],[0.02360273],[0.00595916],[0.00587166],[-0.00143298],[-0.0019266],[0.00365164]
3,Statistici,[0.02276427],[-0.00050032],[0.04481152],[0.00710991],[-0.00618336],[0.00165096],[0.00917636],[-0.02183792],[-0.00394505],...,[-0.00588245],[-0.01281611],[0.01174118],[-0.00255709],[0.02007497],[0.00919405],[-0.00659355],[0.00021914],[-0.00382449],[0.00017763]
4,Commessi delle vendite al minuto,[0.02147509],[-0.00927484],[0.02051596],[0.00021512],[-0.00348184],[-0.00197966],[0.0037389],[-0.00521202],[0.00364424],...,[-0.01851345],[-0.01068269],[0.00928148],[0.01188362],[0.00296454],[0.00482857],[0.00694664],[-0.01691168],[-0.00841968],[0.00035827]


In [184]:
for i in range(2,302):
    dati_with_embedd[i] = dati_with_embedd[i].apply(lambda x: float(x[1:-1]))

In [185]:
dati_with_embedd

,1,2,3,4,5,6,7,8,9,10,...,292,293,294,295,296,297,298,299,300,301
0,Commessi delle vendite al minuto,0.029060,0.001909,0.031447,0.001542,-0.003907,-0.006787,0.009617,-0.009915,-0.009480,...,-0.012308,-0.006692,-0.004176,0.009623,0.035008,-0.000217,0.012981,-0.019019,0.006348,-0.002230
1,Commessi delle vendite al minuto,0.014807,-0.003312,0.044902,0.001014,-0.009834,0.003029,0.002283,0.003144,-0.038378,...,-0.021458,-0.018622,0.007902,0.016694,0.009198,0.010742,-0.008505,-0.017598,-0.001319,0.001052
2,Camerieri di ristorante,0.014255,-0.013534,0.041608,0.005601,-0.008877,-0.008680,0.010965,-0.005354,-0.031990,...,-0.013368,-0.014966,-0.001656,0.008230,0.023603,0.005959,0.005872,-0.001433,-0.001927,0.003652
3,Statistici,0.022764,-0.000500,0.044812,0.007110,-0.006183,0.001651,0.009176,-0.021838,-0.003945,...,-0.005882,-0.012816,0.011741,-0.002557,0.020075,0.009194,-0.006594,0.000219,-0.003824,0.000178
4,Commessi delle vendite al minuto,0.021475,-0.009275,0.020516,0.000215,-0.003482,-0.001980,0.003739,-0.005212,0.003644,...,-0.018513,-0.010683,0.009281,0.011884,0.002965,0.004829,0.006947,-0.016912,-0.008420,0.000358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,Tecnici web,0.011746,-0.013130,0.017576,0.015523,0.000140,0.002014,0.024772,-0.006235,0.009625,...,-0.010760,-0.016031,0.003890,0.023780,0.022583,0.004480,-0.000858,0.008051,0.012189,-0.002860
646,Camerieri di ristorante,0.017056,0.001730,0.028818,0.005725,-0.000194,-0.011580,0.011742,-0.002532,-0.013934,...,-0.017217,-0.021262,0.004697,0.014992,0.010189,0.000257,0.002670,-0.007782,-0.005347,0.001831
647,Commessi delle vendite al minuto,0.025454,-0.004507,0.032442,0.003334,-0.005258,-0.015966,0.017312,0.000436,-0.012918,...,-0.017366,-0.012591,0.006582,0.018569,0.017397,0.019034,0.020793,0.000829,0.011212,0.001634
648,Statistici,0.008747,-0.003246,0.059828,0.002654,-0.006033,-0.011740,0.008239,-0.016104,-0.023398,...,-0.010580,-0.015682,0.002475,0.004114,0.030707,0.009578,-0.001114,-0.003617,0.003038,-0.001419


In [202]:
Role_dictionary = pd.Series(dati_with_embedd[1].unique()).to_dict()
Role_dictionary = dict([(value, key) for key, value in Role_dictionary.items()])
Role_dictionary

{'Commessi delle vendite al minuto': 0,
 'Camerieri di ristorante': 1,
 'Statistici': 2,
 'Tecnici web': 3,
 'Cuochi in alberghi e ristoranti': 4,
 'Elettromeccanici': 5}

In [204]:
dati_with_embedd['class_encoded'] = dati_with_embedd[1].apply(lambda x: Role_dictionary.get(x,'Unknown'))
dati_with_embedd

,1,2,3,4,5,6,7,8,9,10,...,293,294,295,296,297,298,299,300,301,class_encoded
0,Commessi delle vendite al minuto,0.029060,0.001909,0.031447,0.001542,-0.003907,-0.006787,0.009617,-0.009915,-0.009480,...,-0.006692,-0.004176,0.009623,0.035008,-0.000217,0.012981,-0.019019,0.006348,-0.002230,0
1,Commessi delle vendite al minuto,0.014807,-0.003312,0.044902,0.001014,-0.009834,0.003029,0.002283,0.003144,-0.038378,...,-0.018622,0.007902,0.016694,0.009198,0.010742,-0.008505,-0.017598,-0.001319,0.001052,0
2,Camerieri di ristorante,0.014255,-0.013534,0.041608,0.005601,-0.008877,-0.008680,0.010965,-0.005354,-0.031990,...,-0.014966,-0.001656,0.008230,0.023603,0.005959,0.005872,-0.001433,-0.001927,0.003652,1
3,Statistici,0.022764,-0.000500,0.044812,0.007110,-0.006183,0.001651,0.009176,-0.021838,-0.003945,...,-0.012816,0.011741,-0.002557,0.020075,0.009194,-0.006594,0.000219,-0.003824,0.000178,2
4,Commessi delle vendite al minuto,0.021475,-0.009275,0.020516,0.000215,-0.003482,-0.001980,0.003739,-0.005212,0.003644,...,-0.010683,0.009281,0.011884,0.002965,0.004829,0.006947,-0.016912,-0.008420,0.000358,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,Tecnici web,0.011746,-0.013130,0.017576,0.015523,0.000140,0.002014,0.024772,-0.006235,0.009625,...,-0.016031,0.003890,0.023780,0.022583,0.004480,-0.000858,0.008051,0.012189,-0.002860,3
646,Camerieri di ristorante,0.017056,0.001730,0.028818,0.005725,-0.000194,-0.011580,0.011742,-0.002532,-0.013934,...,-0.021262,0.004697,0.014992,0.010189,0.000257,0.002670,-0.007782,-0.005347,0.001831,1
647,Commessi delle vendite al minuto,0.025454,-0.004507,0.032442,0.003334,-0.005258,-0.015966,0.017312,0.000436,-0.012918,...,-0.012591,0.006582,0.018569,0.017397,0.019034,0.020793,0.000829,0.011212,0.001634,0
648,Statistici,0.008747,-0.003246,0.059828,0.002654,-0.006033,-0.011740,0.008239,-0.016104,-0.023398,...,-0.015682,0.002475,0.004114,0.030707,0.009578,-0.001114,-0.003617,0.003038,-0.001419,2


In [210]:
X_train, X_test, y_train, y_test = train_test_split(dati_with_embedd.iloc[:,1:-1],
                                                    dati_with_embedd['class_encoded'],
                                                   test_size=0.20, random_state=42)

In [211]:
X_train

,2,3,4,5,6,7,8,9,10,11,...,292,293,294,295,296,297,298,299,300,301
333,0.000346,0.001877,0.031898,0.005943,0.002801,-0.001939,0.006199,0.000935,-0.009053,-0.022022,...,-0.008293,-0.017333,-0.003172,0.004917,0.023041,0.013281,0.000408,-0.001082,-0.000905,0.007301
29,0.016215,-0.011549,0.046444,-0.000137,-0.008235,-0.014681,0.006674,-0.002788,-0.023442,-0.028210,...,-0.023596,-0.016055,-0.000256,0.014712,0.024432,0.008953,0.007894,-0.002156,0.006079,-0.002797
553,0.017320,-0.007010,0.036398,0.005652,0.006854,-0.002017,0.007178,-0.009284,-0.007675,-0.010309,...,-0.019035,-0.022076,-0.000432,-0.002356,0.014846,0.007087,-0.006403,-0.003701,0.001200,-0.006133
286,0.013604,-0.002207,0.037749,0.006546,-0.004303,-0.004935,0.009516,0.001298,-0.018662,-0.020180,...,-0.012129,-0.025686,-0.000954,0.013175,0.024167,0.003766,-0.003972,-0.014005,-0.009986,0.011158
425,0.016459,-0.003742,0.049261,0.003421,-0.013783,-0.009349,0.012221,0.002937,-0.025421,-0.019560,...,-0.010706,-0.012061,0.005089,0.012468,0.026418,0.000179,0.003139,-0.008322,-0.006476,0.000616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.022248,-0.006556,0.035672,0.007364,-0.010853,-0.006340,0.020071,-0.007227,-0.010703,-0.020800,...,-0.015115,-0.007712,0.014822,0.009377,0.019994,0.002865,-0.006801,-0.003684,-0.001787,-0.005315
106,0.017211,0.001202,0.043934,0.014656,-0.010362,-0.007177,0.022121,-0.013438,-0.022653,-0.022589,...,-0.025300,-0.008651,-0.001973,0.012629,0.046947,-0.007542,0.013966,-0.008307,-0.007256,-0.011800
270,0.018759,-0.001859,0.030258,-0.001354,-0.010810,-0.011385,0.000365,-0.001584,-0.009336,-0.022520,...,-0.011699,-0.012208,0.004457,0.013950,0.006542,0.000180,0.007677,-0.011055,-0.000882,0.000782
435,0.021101,-0.001854,0.021830,0.002078,-0.002305,-0.010857,0.007945,-0.014890,-0.027756,-0.014522,...,-0.017064,-0.015031,0.002750,0.017042,0.012082,0.012789,0.007388,-0.015853,-0.001595,0.008411


In [229]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y_train)
y

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [262]:
from tensorflow.keras import Sequential, initializers
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
# model = Sequential()
# model.add(Dense(150, input_dim=300, activation='relu',kernel_initializer='he_uniform'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(5, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model = Sequential()
model.add(Dense(300, input_dim=300, activation='relu', kernel_initializer=initializers.RandomNormal(mean=0., stddev=1.)))
model.add(Dropout(0.1))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='softmax'))
# compile model
opt = SGD(lr=0.005, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y, epochs=150, batch_size=16)

Epoch 1/150
520/520 [==============================] - 1s 2ms/step - loss: 1.7706 - acc: 0.2212
Epoch 2/150
520/520 [==============================] - 0s 204us/step - loss: 1.7001 - acc: 0.3096
Epoch 3/150
520/520 [==============================] - 0s 233us/step - loss: 1.6066 - acc: 0.4096
Epoch 4/150
520/520 [==============================] - 0s 318us/step - loss: 1.4754 - acc: 0.4462
Epoch 5/150
520/520 [==============================] - 0s 204us/step - loss: 1.2998 - acc: 0.4923
Epoch 6/150
520/520 [==============================] - 0s 211us/step - loss: 1.1597 - acc: 0.5385
Epoch 7/150
520/520 [==============================] - 0s 197us/step - loss: 1.0408 - acc: 0.5904
Epoch 8/150
520/520 [==============================] - 0s 189us/step - loss: 0.9585 - acc: 0.6192
Epoch 9/150
520/520 [==============================] - 0s 271us/step - loss: 0.9085 - acc: 0.6135
Epoch 10/150
520/520 [==============================] - 0s 232us/step - loss: 0.8050 - acc: 0.6942
Epoch 11/150
520/520 

In [263]:
testy = to_categorical(y_test)
_, test_acc = model.evaluate(X_test, testy, verbose=0)

In [264]:
test_acc

0.8923076923076924

## testing with real job vacancies

In [304]:
unseen_data = pd.DataFrame({'Job_Description':['Randstad Technical di Vimercate seleziona un manutentore per inserimento in somministrazione presso azienda del settore gomma plastica. La risorsa si occuperà della manutenzione ordinaria e straordinaria di presse a iniezione idrauliche  ed elettriche, di attrezzature elettromeccaniche e degli impianti del plant produttivo. Requisiti: diploma o qualifica in ambito meccanico o elettromeccanico; esperienza nella mansione di almeno 3 anni; disponibilità al lavoro sia a giornata che su tre turni; flessibilità, precisione, capacità organizzative, proattività']})
unseen_data

,Job_Description
0,Randstad Technical di Vimercate seleziona un m...


In [305]:
unseen_data['word_embedding'] = unseen_data.Job_Description.apply(lambda x: ft.get_sentence_vector(x.replace('\n',' ')))
unseen_data

,Job_Description,word_embedding
0,Randstad Technical di Vimercate seleziona un m...,"[0.0149618415, -0.0011330143, 0.029707536, 0.0..."


In [306]:
unseen_data.word_embedding = unseen_data.word_embedding.apply(lambda x: ','.join(str(i) for i in np.split(x,300)))
matrix_embedding = unseen_data.word_embedding.str.split(',',expand = True)

In [307]:
for i in range(0,300):
    matrix_embedding[i] = matrix_embedding[i].apply(lambda x: float(x[1:-1]))

In [308]:
matrix_embedding

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.014962,-0.001133,0.029708,0.00321,0.003244,-0.002932,0.020201,-0.005512,-0.017781,-0.016539,...,-0.02555,-0.014651,-0.00742,0.006872,0.032563,-0.004578,0.007414,-0.00815,-0.010927,-0.016459


In [309]:
predicted_label = int(model.predict_classes(matrix_embedding))
predicted_label

5

In [310]:
def get_key(value,dictionary):
    for key,val in dictionary.items():
        if value == val:
            return key

In [311]:
get_key(predicted_label,Role_dictionary)

'Elettromeccanici'